In [1]:
import tensorflow as tf
import numpy as np
from util.data_management import load_data
from util.feature import stack_feature, display_feature, get_mel_spectrogram_with_librosa
from util.data_generator import DataGenerator
from tensorflow.python.client import device_lib
import time
from tensorflow.python.keras.callbacks import TensorBoard
from sklearn.metrics import confusion_matrix
from util.metric import plot_confusion_matrix, plot_to_image
import os
from easydict import EasyDict

In [ ]:
print(device_lib.list_local_devices())

In [2]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    tf.config.experimental.set_memory_growth(gpus[0], True)
  except RuntimeError as e:
    # 프로그램 시작시에 메모리 증가가 설정되어야만 합니다
    print(e)

In [3]:
args = EasyDict({
    'MODEL_NAME' : 'CNN',
    'BATCH_SIZE' : 64,
    'SHUFFLE' : True,
    'SEED' : 555,
    'NUM_CLASSES' : 1,
    'INPUT_SHAPE' : (39, 532), # Width, Height
    'LOG_PATH' : 'training_log',
    'LEARNING_RATE' : 1e-3,
    'NUM_MEL_BIN' : 39,
    'SPLIT' : (0.9, 0.1, 0.0), # Train, Validation, Test
    'EPOCH' : 100,
    'NUM_LIMIT' : 4000
})

DATASET_PATH = r'C:\Users\VIP444\Documents\Github\AnomalyDetection\feature.json'

try:
    os.mkdir(args.LOG_PATH)
except FileExistsError as e:
    pass

In [ ]:
audio_data_generator = DataGenerator(
                                        DATASET_PATH,
                                        seed=args.SEED,
                                        split=args.SPLIT,
                                        num_limit=args.NUM_LIMIT,
                                        shuffle=True,
                                        batch_size=args.BATCH_SIZE,
                                        caching=True,
                                    )

In [ ]:
train_dataset = audio_data_generator(train_mode='train')
val_dataset = audio_data_generator(train_mode='val') 

In [ ]:
input_shape = train_dataset.element_spec[0].shape[1:]
label_shape = train_dataset.element_spec[1].shape
# label_names = np.array(tf.io.gfile.listdir(f'{DATASET_PATH}'))
# num_labels = len(label_names)
print(input_shape, label_shape)

In [2]:
from tensorflow.python.keras.layers import Dense, Conv2D, MaxPool2D, GlobalAvgPool2D, BatchNormalization
from tensorflow.python.keras import Sequential
input_shape = (39,532,4)

model = Sequential()
model.add(Conv2D(input_shape=input_shape, filters=32, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=32, kernel_size=(3,3),padding="same", activation="relu"))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
model.add(Conv2D(filters=64, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=64, kernel_size=(3,3), padding="same", activation="relu"))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
model.add(Conv2D(filters=1024, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=1024, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=1024, kernel_size=(3,3), padding="same", activation="relu"))
model.add(BatchNormalization())
model.add(GlobalAvgPool2D())
model.add(Dense(units=256, activation="relu"))
model.add(Dense(units=128, activation="relu"))
model.add(Dense(units=64, activation="relu"))
model.add(Dense(units=1, activation="sigmoid"))

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 39, 532, 32)       1184      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 39, 532, 32)       9248      
_________________________________________________________________
batch_normalization (BatchNo (None, 39, 532, 32)       128       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 19, 266, 32)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 19, 266, 64)       18496     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 19, 266, 64)       36928     
_________________________________________________________________
batch_normalization_1 (Batch (None, 19, 266, 64)      

In [ ]:
start_time = time.strftime("%Y-%m-%d-%H%M%S", time.localtime(time.time()))

EXPERIMENT_DIR_NAME = f'{args.LOG_PATH}/{start_time}-{args.MODEL_NAME}'
tensorboard = TensorBoard(log_dir=f'{EXPERIMENT_DIR_NAME}/tensorboard_logs', profile_batch=0)
file_writer_cm = tf.summary.create_file_writer(logdir=f'{EXPERIMENT_DIR_NAME}/tensorboard_logs')

In [ ]:
with open(f'{EXPERIMENT_DIR_NAME}/params.txt', 'w') as file:
    file.write(f'MODEL NAME : {args.MODEL_NAME}\n')
    file.write(f'DATASET_PATH : {DATASET_PATH}\n')
    file.write(f'BATCH SIZE : {args.BATCH_SIZE}\n')
    file.write(f'SHUFFLE : {args.SHUFFLE}\n')
    file.write(f'SEED : {args.SEED}\n')
    file.write(f'WIDTH HEIGHT : {args.INPUT_SHAPE}\n')
    file.write(f'SPLIT : {args.SPLIT}\n')
    file.write(f'LEARNING_RATE : {args.LEARNING_RATE}\n')
    file.write(f'NUM_MEL_BIN : {args.NUM_MEL_BIN}\n')

    for key in args.keys():
        file.write(f'{key.upper()} : {args[key]}\n')
    

In [ ]:
num_labels = 2
label_names = ['normal', 'abnoraml']

def log_confusion_matrix(epoch, logs):
    # Use the model to predict the values from the validation dataset.
    test_pred_raw = model.predict(val_dataset)
    if num_labels > 2:
        test_pred = np.argmax(test_pred_raw, axis=1)
        # Calculate the confusion matrix.
        cm = confusion_matrix(np.argmax(audio_data_generator.validation_labels, axis=1), test_pred)
    else:
        test_pred = np.round(tf.nn.sigmoid(test_pred_raw))
        # Calculate the confusion matrix.
        cm = confusion_matrix(audio_data_generator.validation_labels, test_pred)

    # Log the confusion matrix as an image summary.
    figure = plot_confusion_matrix(cm, class_names=label_names)
    cm_image = plot_to_image(figure)

    # Log the confusion matrix as an image summary.
    with file_writer_cm.as_default():
        tf.summary.image("Confusion Matrix", cm_image, step=epoch)

In [ ]:
from tensorflow.python.keras.optimizer_v2.adam import Adam

if num_labels > 2:
    loss = tf.keras.losses.CategoricalCrossentropy(from_logits=True)
else:
    loss = tf.keras.losses.BinaryCrossentropy(from_logits=True)

model.compile(
    optimizer=Adam(learning_rate=args.LEARNING_RATE),
    loss=loss,
    metrics=['accuracy'],
)

In [ ]:
callbacks = [
    tf.keras.callbacks.EarlyStopping(verbose=1, patience=7),
    tensorboard,
    tf.keras.callbacks.LambdaCallback(on_epoch_end = log_confusion_matrix),
    tf.keras.callbacks.ModelCheckpoint(f'{EXPERIMENT_DIR_NAME}/model-{{epoch:02d}}', save_best_only=True, monitor='val_loss', mode='min'),
    tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.8, patience=6, verbose=1, min_lr=args.LEARNING_RATE * 1e-1),
    tf.keras.callbacks.CSVLogger(f'{EXPERIMENT_DIR_NAME}/train_log.csv', separator=',', append=True),
]

In [ ]:
history = model.fit(train_dataset, 
                    validation_data=val_dataset, 
                    callbacks=callbacks, 
                    epochs=100)

In [4]:
model = tf.keras.models.load_model(r'C:\Users\VIP444\Documents\Github\AnomalyDetection\training_log\2022-03-15-215804-CNN\model-05')

In [5]:
import csv

class EvaluateCSVLogger(tf.keras.callbacks.Callback):
    def __init__(self, filename, seperator=',', append=True):
        self.filename = filename
        self.seperator= seperator
        self.append = append

    def on_test_begin(self, logs=None):
        self.csvfile = open(self.filename, 'w' if not self.append else 'a', newline='')
        self.csvwriter = csv.writer(self.csvfile, delimiter=self.seperator,
                                    quotechar='|', quoting=csv.QUOTE_MINIMAL)
        print('test begin')

    def on_test_batch_begin(self, batch, logs=None):
        pass

    def on_test_batch_end(self, batch, logs=None):
        # write the contents of the dictionary logs to csv file
        # sample content of logs {'batch': 0, 'size': 2, 'loss': -0.0, 'accuracy': 1.0}
        pass

    def on_test_end(self, logs=None):
        self.csvwriter.writerow(list(logs))
        self.csvwriter.writerow(list(logs.values()))
        self.csvfile.close()
        print('test end')

In [6]:
EXPERIMENT_DIR_NAME = r'C:\Users\VIP444\Documents\Github\AnomalyDetection\training_log\2022-03-15-215804-CNN'

test_audio_generator = DataGenerator(
                                        DATASET_PATH,
                                        seed=args.SEED,
                                        split=(0.0,0.0,1.0),
                                        num_limit=args.NUM_LIMIT,
                                        shuffle=True,
                                        batch_size=args.BATCH_SIZE,
                                        caching=True,
                                    )

test_dataset = test_audio_generator(train_mode='test')
predict_raw = model.predict(test_dataset,verbose=1)

model.evaluate(
    test_dataset,
    callbacks=[EvaluateCSVLogger(f'{EXPERIMENT_DIR_NAME}/test_log.csv', seperator=',', append=True)]
)

19/19 [==============================] - 6s 26ms/step
test begin
 4/19 [=====>........................] - ETA: 0s - loss: 6.1963e-15 - accuracy: 1.0000

C:\Users\VIP444\Anaconda3\envs\keras\lib\site-packages\tensorflow\python\keras\backend.py:5017: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  '"`binary_crossentropy` received `from_logits=True`, but the `output`'


19/19 [==============================] - 1s 20ms/step - loss: 23.2156 - accuracy: 0.3457
test end


[23.215585708618164, 0.34572169184684753]

In [10]:
print(len(test_audio_generator.test_paths))

1157


In [7]:
from tqdm import tqdm
from numpy import argmax
import numpy as np
from sklearn.metrics import roc_curve

label_names = ['abnoraml', 'normal']

def predict_logger(output, original_path,original, predict, label_names, error_only: bool = False, threshold=0.5):

    with open(output, 'w', encoding='utf-8', newline='') as csvfile:
        csv_writer = csv.writer(csvfile)

        if original is not None :
            csv_writer.writerow(['file path','file length', 'original label', 'predict label', *label_names])

            for path, original_label, predict_logit in tqdm(zip(original_path, original, predict)):

                if len(label_names) >= 3:
                    predict_label_name = label_names[np.argmax(predict_logit, axis=0)]
                else:
                    predict_label_name = label_names[int(predict_logit > threshold)]

                original_label_name = label_names[original_label]

                if error_only and predict_label_name == original_label_name:
                    continue

                csv_writer.writerow([original_label_name, predict_label_name, *(list(predict_logit))])

        else:

            csv_writer.writerow(['file path','file length','predict label', *label_names])

            for path, predict_logit in tqdm(zip(original_path, predict)):
                basename = os.path.basename(path)

                if len(label_names) >= 3:
                    predict_label_name = label_names[np.argmax(predict_logit, axis=0)]
                else:
                    predict_label_name = label_names[int(predict_logit > threshold)]

                csv_writer.writerow([basename,predict_label_name, *predict_logit])


predict = tf.nn.sigmoid(predict_raw)

y_true = np.array(test_audio_generator.test_labels)
y_pred = np.array(predict)

# calculate roc curves
fpr, tpr, thresholds = roc_curve(y_true, y_pred)
# get the best threshold
J = tpr - fpr
ix = argmax(J)
best_thresh = thresholds[ix]

print(best_thresh)

predict_logger(f'{EXPERIMENT_DIR_NAME}/test-class2.csv', 
                test_audio_generator.test_paths, 
                test_audio_generator.test_labels,
                predict,
                label_names,
                threshold=best_thresh
                )

with open(f'{EXPERIMENT_DIR_NAME}/params.txt', 'a') as file:
    file.write(f'BEST TRESHOLD : {best_thresh}\n')

1.7310586


1157it [00:00, 2129.45it/s]


In [8]:
from sklearn.metrics import confusion_matrix
from util.metric import plot_confusion_matrix, plot_to_image
import matplotlib.pyplot as plt

if len(['abnormal', 'normal']) >= 3:
    test_pred = np.argmax(predict, axis=1)
    # Calculate the confusion matrix.
    cm = confusion_matrix(np.argmax(test_audio_generator.test_labels, axis=1), test_pred)
else:
    test_pred = np.array(predict > best_thresh ,dtype=np.int64)
    # Calculate the confusion matrix.
    cm = confusion_matrix(test_audio_generator.test_labels, test_pred)

# Log the confusion matrix as an image summary.
figure = plot_confusion_matrix(cm, class_names=['abnormal', 'normal'])
cm_image = plot_to_image(figure)

plt.imsave(f'{EXPERIMENT_DIR_NAME}/test-confusion.png', tf.squeeze(cm_image, axis=0).numpy())

In [9]:
print(min(test_audio_generator.test_labels))

0
